In [9]:
import pandas as pd
import numpy as np


In [10]:
# для всех норм по ТПГ нужно объявить эту переменную
row_number=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,usecols=[5])
skip_row=row_number.ЦОК.tolist()
skip_row_n=skip_row.index('подгруппа')-2

In [12]:
#СЧИТАЕТ КОЛИЧЕСТВО ОТКРЫТЫХ ЗАКРЫТЫХ 
df=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2)
df1=df[["статус","Номер из NAV","Адрес магазина","дата планового открытия"]]
df1=df1[~df1.статус.isin(['Закрыт','закрыт'])]
df1.groupby(by='статус').count()

,Номер из NAV,Адрес магазина,дата планового открытия
статус,,,
Отгружено,3,3,3
открыт,888,888,871
склад,21,21,21


In [13]:
#СЧИТАЕТ КОЛИЧЕСТВО ОТКРЫТЫХ ЗАКРЫТЫХ иф/бк
a=[2,3,18]+[*range(28,41)]
a_shops=df.iloc[1:, lambda df: a]
del a_shops['ИФ браслеты'], a_shops['ИФ аксессуары']
a_shops=a_shops[a_shops.статус.isin(['открыт','Отгружено','склад'])]
a_shops['сумма']=a_shops.iloc[:,3:].sum(axis=1)
a_shops=a_shops[a_shops['сумма'] >0]
a_shops[['статус','Номер из NAV']].groupby(by=['статус']).count()

,Номер из NAV
статус,
Отгружено,3
открыт,861
склад,21


In [14]:
#СОХРАНЯЕТ СПИСОК НОВЫХ МАГОВ

new_shops=df1[(df1.статус == 'Отгружено')|(df1.статус == 'склад')]
new_shops=new_shops.drop(columns='статус').sort_values('дата планового открытия')
new_shops.set_index('Номер из NAV',drop=True, inplace=True)
new_shops.to_excel(r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\new_shops.xlsx')

#ДОСТАЕТ НОРМЫ ПО магам иф/бк

norm_all=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,nrows=skip_row_n)
list=norm_all.columns.tolist()
cols=[list[list.index('Название из NAV')]]+list[list.index(1001):list.index(1093)+1]+list[list.index(1501):list.index(1593)+1]
opened=a_shops[['Номер из NAV','статус']]
norm_all=norm_all[cols]
merged=pd.merge(opened,norm_all,left_on='Номер из NAV', right_on='Название из NAV',how = 'left')
del merged['Название из NAV'],merged['Номер из NAV']
merged=merged.groupby(by='статус').sum()
merged=merged.T
merged=merged.assign(sum_all=merged['Отгружено']+merged['открыт']+merged['склад'])
del merged['Отгружено'],merged['склад']
merged.to_excel(r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\нормы_по_тпг.xlsx')


In [4]:
#ДОСТАЕТ НОРМЫ ПО БУКВАМ
norm=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,nrows=skip_row_n)
list=norm.columns.tolist()
cols=[list[list.index('Название из NAV')]]+list[list.index(1079):list.index(1079)+3]+list[list.index(1582):list.index(1582)+3]
norm=norm[cols]
norm=norm.assign(sum_if=norm[1079]+norm[1080]+norm[1081],sum_bk=norm[1582]+norm[1583]+norm[1584])
norm.set_index('Название из NAV',drop=True, inplace=True)
norm.to_excel(r'C:\Остатки\norm_b.xlsx')

In [7]:
skip_row_n

1397

In [5]:
#ДОСТАЕТ НОРМЫ ПО ЗОДИАКАМ
normb=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,nrows=skip_row_n)
listb=normb.columns.tolist()
colsb=[listb[listb.index('Название из NAV')]]+listb[listb.index(1082):listb.index(1082)+3]+listb[listb.index(1585):listb.index(1585)+3]
normb=normb[colsb]
normb=normb.assign(sum_if1=normb[1082]+normb[1083]+normb[1084],sum_bk1=normb[1585]+normb[1586]+normb[1587])
normb.set_index('Название из NAV',drop=True, inplace=True)
normb=normb[[1082,1083,1084,'sum_if1',1585,1586,1587,'sum_bk1']]
normb.to_excel(r'C:\Остатки\norm_z.xlsx')


In [8]:
#ВЫДАЕТ СПИСОК АКТУАЛЬНЫХ МАГОВ ИФ\БК С НОРМАМИ ПО ТГ
df=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2)
a=[2,3,18]+[*range(28,41)]
a_shops=df.iloc[1:, lambda df: a]
del a_shops['ИФ браслеты'], a_shops['ИФ аксессуары']
a_shops=a_shops[a_shops.статус.isin(['открыт','Отгружено','склад'])]
a_shops['сумма']=a_shops.iloc[:,3:].sum(axis=1)
a_shops=a_shops[a_shops['сумма'] >0]
del a_shops['статус'], a_shops['сумма']
cols = a_shops.columns.tolist()
a_shops=a_shops[cols[:4]+[cols[6],cols[5],cols[4]]+cols[7:10]+[cols[12],cols[11],cols[10]]]
a_shops.set_index('Номер из NAV',drop=True, inplace=True)
a_shops.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\актуальные маги.xlsx'
)

In [94]:
#ФОРМИРУЕТ СПИСОК МАГОВ ДЛЯ ФАЙЛА ЗАКРЫТЬ МАГИ
df=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2)
a=[2,3,18]+[*range(28,41)]
a_shops=df.iloc[1:, lambda df: a]
del a_shops['ИФ браслеты'], a_shops['ИФ аксессуары']
a_shops=a_shops[a_shops.статус.isin(['открыт','Отгружено','склад'])]
a_shops['сумма']=a_shops.iloc[:,3:].sum(axis=1)
a_shops=a_shops[a_shops['сумма'] >0]
del a_shops['статус'], a_shops['сумма']
col1=a_shops['Номер из NAV'].tolist()*11
col1=sorted(col1)
[int(x) for x in col1]
col2 = a_shops.columns.tolist()
col2 =col2[2:4]+[col2[6],col2[5],col2[4]]+col2[7:10]+[col2[12],col2[11],col2[10]]
col2=col2*(int(len(col1)/11))
to_close=pd.DataFrame({'num':col1,'tn':col2})
to_close.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\для закрыть.xlsx'
)


In [15]:
# СПИСОК НОРМ ПО ТГ

#читает файл с реестром
df=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2)

#вынимает нужные две строки и сохраняет
rep_tn=df.iloc[0:1,28:73]
rep_tn.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Татьяна ИФ.БК\РЕЙТИНГИ\Рейтинг ТГ и ТН\123.xlsx')

In [254]:
mus=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\мусульманкэ.xlsx',
    sheet_name='Лист1')
mus

,Название из NAV,1055,1056,1057,1058,1059,1060,1061,1062,1063,...,1563,1564,1565,1566,1567,1568,1569,1570,1571,1572
0,789,2.038015,6.793384,4.755369,15.205548,11.310957,5.21276,52.754625,45.873587,16.055755,...,2.009433,62.973673,44.081571,18.892102,12.846861,14.988005,14.988005,13.854208,12.314852,4.618069
1,3081,1.972311,6.574369,4.602059,7.602774,5.655479,2.60638,51.053849,44.394651,15.538128,...,2.326864,72.921659,51.045161,21.876498,14.876287,17.355668,17.355668,16.042765,14.260235,5.347588
2,3272,2.065490,6.884968,4.819478,0.000000,0.000000,0.00000,53.465830,46.492026,16.272209,...,2.561425,80.272590,56.190813,24.081777,16.375904,19.105221,19.105221,17.659970,15.697751,5.886657
3,3653,1.621691,5.405636,3.783945,3.801387,2.827739,1.30319,41.977949,36.502564,12.775898,...,1.476076,46.258807,32.381165,13.877642,9.436967,11.009795,11.009795,10.176938,9.046167,3.392313
4,3954,1.336775,4.455918,3.119142,7.602774,5.655479,2.60638,34.602825,30.089413,10.531294,...,1.144088,35.854610,25.098227,10.756383,7.314473,8.533551,8.533551,7.888014,7.011568,2.629338
5,4345,2.542142,8.473807,5.931665,0.000000,0.000000,0.00000,65.804098,57.220955,20.027334,...,1.582057,49.580129,34.706090,14.874039,10.114529,11.800284,11.800284,10.907628,9.695670,3.635876


In [282]:
col2=mus.columns.tolist()[1:]*len(mus['Название из NAV'])

Название из NAV    3081.000000
1055                  1.972311
1056                  6.574369
1057                  4.602059
1058                  7.602774
1059                  5.655479
1060                  2.606380
1061                 51.053849
1062                 44.394651
1063                 15.538128
1558                  2.647989
1559                  5.146186
1560                  0.341193
1561                 39.002040
1562                 36.454199
1563                  2.326864
1564                 72.921659
1565                 51.045161
1566                 21.876498
1567                 14.876287
1568                 17.355668
1569                 17.355668
1570                 16.042765
1571                 14.260235
1572                  5.347588
Name: 1, dtype: float64

In [260]:
mus[['']]

KeyError: "['str(1056)'] not in index"

In [228]:
mus.iloc[1:,1:2]


,1055
1,1.972311
2,2.065490
3,1.621691
4,1.336775
5,2.542142


0.0046296296296296285